# Hand digit classifier 
---
## Incremental network quantization

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from skimage import io

In [ ]:
def getBounderyExponents(W, b):
    s = torch.max(torch.abs(W)).item()
    n1 = np.floor(np.log2(4*(s/3)))
    n2 = n1 + 1 - (2**(b - 1))/2
    return n1, n2

def getQuantizationMask(W, percentage, T):
    w = W.view(-1)
    t = T.view(-1)
    idx = t == 1

    numberOfWeights = w.size(dim=0)
    numberOfQWeights = int(percentage*numberOfWeights - t[idx].size(dim=0))

    t_aux = torch.Tensor(np.ones_like(T)).view(-1)
    w = w*(t_aux - t)
    w = torch.abs(w)
    sorted_w, indices_w = w.sort()
    print(w, sorted_w, indices_w)
    t[indices_w[-numberOfQWeights:]] = 1
    
    return t.view(T.size())

def indexTensor(tensor, map):
    pass

In [ ]:
W = torch.tensor(np.array([
    [0.01, 0.02, -0.2, 0.04, 0.33],
    [0.17, -0.42, -0.33, 0.02, -0.05], 
    [0.02, 0.83, -0.03, 0.03, 0.06],
    [-0.9, 0.07, 0.11, 0.87, -0.36], 
    [-0.73, 0.41, 0.42, 0.39, 0.47]]))
bit_length = 4
n1, n2 = getBounderyExponents(W, bit_length)
print(n1, n2)



In [23]:
percentage = 0.5
W = torch.Tensor(np.array([[-1, 2, -2], [2, 3, -0.5], [4, 3, 2]]))
T = torch.Tensor(np.zeros_like(W))
w = W.view(-1)
t = T.view(-1)
idx = t == 1

numberOfWeights = w.size(dim=0)
numberOfQWeights = int(percentage*numberOfWeights - t[idx].size(dim=0))

t_aux = torch.Tensor(np.ones_like(T)).view(-1)
w = w*(t_aux - t)
w = torch.abs(w)
sorted_w, indices_w = w.sort()
print(w, sorted_w, indices_w)
t[indices_w[-numberOfQWeights:]] = 1

T = t.view(T.size())
print(W)
print(T)

tensor([1.0000, 2.0000, 2.0000, 2.0000, 3.0000, 0.5000, 4.0000, 3.0000, 2.0000]) tensor([0.5000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 3.0000, 3.0000, 4.0000]) tensor([5, 0, 1, 2, 3, 8, 4, 7, 6])
tensor([[-1.0000,  2.0000, -2.0000],
        [ 2.0000,  3.0000, -0.5000],
        [ 4.0000,  3.0000,  2.0000]])
tensor([[0., 0., 0.],
        [0., 1., 0.],
        [1., 1., 1.]])
